<a href="https://www.bigdatauniversity.com"><img src = "https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width = 300, align = "center"></a>

<h1 align=center><font size = 5>Laboratorio: Cómo crear tablas e insertar y consultar datos </font></h1>

# Introducción

Esta libreta enseña cómo acceder a su instancia de base de datos, crear tablas e insertar y consultar datos con Python. Para ello, debe seguir los pasos a continuación: 
1. Importe la biblioteca de Python `ibm_db`
1. Identifique e ingrese las credenciales para conectarse a la base de datos
1. Cree la conexión a la base de datos 
1. Cree una tabla
1. Inserte datos en la tabla 
1. Consulte datos de la tabla
1. Importe el conjunto de resultados a un DataFrame de Pandas
1. Cierre la conexión a la base de datos


__Nota:__ Siga las instrucciones del documento **Cómo crear su cuenta e instancia de base datos en IBM Cloud** para crear una instancia de servicio de base de datos en Db2 on Cloud.

## Tarea 1: Importe la biblioteca de Python `ibm_db`

La [API ](https://pypi.python.org/pypi/ibm_db/) `ibm_db`  proporciona diversas funciones útiles de Python para acceder a la información de una base de datos en un servidor de IBM® y manipularla; incluye funciones para conectarse a una base de datos, preparar y enviar instrucciones SQL, buscar filas de conjuntos de resultados, llamar procedimientos almacenados, confirmar y revertir transacciones, administrar errores y recuperar metadatos. 

Importe la biblioteca ibm_db a su aplicación de Python


In [1]:
import ibm_db

Una vez que se complete este comando, se cargará la biblioteca `ibm_db` en su libreta. 


## Tarea 2: Identifique las credenciales para conectarse a la base de datos

Para conectarse a una base de datos dashDB o DB2, se necesita la siguiente información: 
* Nombre del controlador
* Nombre de la base de datos
* Nombre del DNS o la dirección IP del host
* Puerto del host 
* Protocolo de conexión 
* ID del usuario
* Contraseña del usuario



__Nota:__ Para obtener las credenciales, consulte las instrucciones del primer laboratorio de este curso.

Debe ingresar las credenciales de la base de datos a continuación. 

Reemplace los siguientes valores de marcador de posición por sus credenciales reales de base de datos. 

Por ejemplo, reemplace "Base de datos" por "BLUDB"



In [2]:
#Reemplace los valores de marcador de posición por sus credenciales reales del servicio de Db2
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_hostname = "dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net"           # e.g.: "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net"
dsn_port = "50000"                    # e.g. "50000" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_uid = "qsl97800"             # e.g. "abc12345"
dsn_pwd = "954gxzq-mk1xbk9z"                 # e.g. "7dBZ3wWt9XN6$o0J"

## Tarea 3: Cree la conexión a la base de datos

La API de Ibm_db usa el controlador del servidor de datos de IBM para las API de ODBC y CLI a fin de conectarse a Informix y Db2 de IBM. 

Cree la conexión a la base de datos 


In [3]:
#Cree una conexión a la base de datos
#NO MODIFIQUE ESTA CELDA. Solo debe EJECUTARLA presionando Shift + Intro
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, 
                       dsn_database, 
                       dsn_hostname, 
                       dsn_port, 
                       dsn_protocol, 
                       dsn_uid, 
                       dsn_pwd)

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Conectado a la base de datos: ", dsn_database, "como el usuario ", dsn_uid, "en el host: ", dsn_hostname)

except:
    print ("No se pudo conectar a la base de datos: ", ibm_db.conn_errormsg() )


Conectado a la base de datos:  BLUDB como el usuario  qsl97800 en el host:  dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net


## Tarea 4: Cree una tabla en la base de datos

En este paso, creará una tabla en la base de datos con los detalles siguientes:

<img src = "https://ibm.box.com/shared/static/ztd2cn4xkdoj5erlk4hhng39kbp63s1h.jpg" align="center">


In [4]:
#Primero elimine la tabla INSTRUCTOR (DOCENTE) por si ya se creó en un intento anterior
dropQuery = "drop table INSTRUCTOR"

#Ahora ejecute la instrucción Eliminar
dropStmt = ibm_db.exec_immediate(conn, dropQuery)

## No se preocupe si se muestra el error siguiente:
Si se muestra una excepción o un error similar al siguiente, la tabla DOCENTE (INSTRUCTOR) no tiene un nombre definido, lo que es correcto. Esto implica que la tabla DOCENTE (INSTRUCTOR) no existe en la tabla; esto es lo que sucederá si no la creó anteriormente. 

Excepción: [IBM][CLI Driver][DB2/LINUXX8664] SQL0204N  "ABC12345.INSTRUCTOR" no es un nombre definido.  SQLSTATE=42704 SQLCODE=-204

In [5]:
#Construct the Create Table DDL statement - replace the ... with rest of the statement
createQuery = "create table INSTRUCTOR(ID INTEGER PRIMARY KEY NOT NULL, FNAME VARCHAR(20), LNAME VARCHAR(20), CITY VARCHAR(20), CCODE CHAR(2))"

#Now fill in the name of the method and execute the statement
createStmt = ibm_db.exec_immediate(conn,createQuery)

Haga doble clic __aquí__ para ver la solución.

<!-- Hint:

createQuery = "create table INSTRUCTOR(ID INTEGER PRIMARY KEY NOT NULL, FNAME VARCHAR(20), LNAME VARCHAR(20), CITY VARCHAR(20), CCODE CHAR(2))"

createStmt = ibm_db.exec_immediate(conn,createQuery)


-->

##  Tarea 5: Inserte datos en la tabla 

En este paso, insertaremos algunas filas de datos en la tabla. 

La tabla DOCENTE (INSTRUCTOR) que creamos en el paso anterior contiene 3 filas de datos: 

<img src="https://ibm.box.com/shared/static/j5yjassxefrjknivfpekj7698dqe4d8i.jpg" align="center">

Comenzaremos por insertar únicamente la primera fila de datos, es decir, la fila que corresponde al docente Rav Ahuja. 


In [6]:
#Cree la instrucción y reemplace … por la instrucción Insertar
insertQuery = "INSERT into INSTRUCTOR values (1, 'Rav', 'Ahuja', 'TORONTO', 'CA')"

#Ejecute la instrucción Insertar 
insertStmt = ibm_db.exec_immediate(conn, insertQuery)

Haga doble clic __aquí__ para ver la solución.

<!-- Hint:

insertQuery = "insert into INSTRUCTOR values (1, 'Rav', 'Ahuja', 'TORONTO', 'CA')"

insertStmt = ibm_db.exec_immediate(conn, insertQuery)


-->

Ahora use una sola consulta para insertar las dos filas de datos restantes. 

In [7]:
#Reemplace … por la instrucción Insertar, que insertará las dos filas de datos restantes
insertQuery2 = "INSERT into INSTRUCTOR values (2, 'Raul', 'Chong', 'Markham', 'CA'), (3, 'Hima', 'Vasudevan', 'Chicago', 'US')"

#Ejecute la instrucción
insertStmt2 = ibm_db.exec_immediate(conn, insertQuery2)

Haga doble clic __aquí__ para ver la solución.

<!-- Hint:

insertQuery2 = "inserte en los valores de INSTRUCTOR (2, 'Raul', 'Chong', 'Markham', 'CA'), (3, 'Hima', 'Vasudevan', 'Chicago', 'US')"

insertStmt2 = ibm_db.exec_immediate(conn, insertQuery2)


-->

##  Tarea 6: Consulte datos de la tabla 

En este paso, recuperaremos datos que insertamos en la tabla INSTRUCTOR (DOCENTE). 

In [9]:
#Cree la instrucción para recuperar todas las filas de la tabla INSTRUCTOR 
selectQuery = "select * from INSTRUCTOR"

#Ejecute la instrucción
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

#Busque el Diccionario (solo para la primera fila) y reemplace … por su código
ibm_db.fetch_both(selectStmt)

{'ID': 1,
 0: 1,
 'FNAME': 'Rav',
 1: 'Rav',
 'LNAME': 'Ahuja',
 2: 'Ahuja',
 'CITY': 'TORONTO',
 3: 'TORONTO',
 'CCODE': 'CA',
 4: 'CA'}

Haga doble clic __aquí__ para ver la solución.

<!-- Hint:

#Cree la instrucción para recuperar todas las filas de la tabla INSTRUCTOR 
selectQuery = "select * from INSTRUCTOR"

#Ejecute la instrucción
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

#Busque el Diccionario (solo para la primera fila)
ibm_db.fetch_both(selectStmt)

-->

In [10]:
#Busque el resto de las filas e imprima el ID y FNAME de estas filas
while ibm_db.fetch_row(selectStmt) != False:
    print(" ID:",  ibm_db.result(selectStmt, 0), " FNAME:",  ibm_db.result(selectStmt, "FNAME"))

 ID: 2  FNAME: Raul
 ID: 3  FNAME: Hima


Haga doble clic __aquí__ para ver la solución.

<!-- Hint:

#Busque el resto de las filas e imprima el ID y FNAME de estas filas
while ibm_db.fetch_row(selectStmt) != False:
    print (" ID:",  ibm_db.result(selectStmt, 0), " FNAME:",  ibm_db.result(selectStmt, "FNAME"))

-->

Extra: Ahora escriba y ejecute una instrucción de actualización que cambie la CIUDAD de Rav a MOOSETOWN. 

In [12]:
#Ingrese su código a continuación
ibm_db.exec_immediate(conn,
                     "UPDATE INSTRUCTOR set CITY='MOOSETOWN' WHERE FNAME='Rav'")

Haga doble clic __aquí__ para ver la solución.

<!-- Hint:

updateQuery = "update INSTRUCTOR set CITY='MOOSETOWN' where FNAME='Rav'"
updateStmt = ibm_db.exec_immediate(conn, updateQuery))

-->

## Tarea 7: Importe datos a Pandas

En este paso, importaremos el contenido de la tabla DOCENTE a un DataFrame de Pandas. 

In [14]:
import pandas
import ibm_db_dbi

In [15]:
#Conexión a Pandas
pconn = ibm_db_dbi.Connection(conn)

In [16]:
#Instrucción de consulta para recuperar todas las filas de la tabla INSTRUCTOR (DOCENTE)
selectQuery = "select * from INSTRUCTOR"

#Importe los resultados de la consulta a un DataFrame de Pandas
pdf = pandas.read_sql(selectQuery, pconn)

#Utilice la instrucción Print únicamente para LNAME en la primera fila del DataFrame de Pandas
pdf.LNAME[0]

'Ahuja'

In [17]:
#Utilice la instrucción Print para todo el DataFrame
pdf

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,MOOSETOWN,CA
1,2,Raul,Chong,Markham,CA
2,3,Hima,Vasudevan,Chicago,US


Una vez que haya importado los datos a un DataFrame de Pandas, puede realizar las operaciones típicas de esta herramienta. 

Por ejemplo, puede usar el método de formato para ver cuántas filas y columnas tiene el DataFrame. 

In [18]:
pdf.shape

(3, 5)

## Tarea 8: Cierre la conexión
Si cerramos la conexión, liberaremos todos los recursos. Recuerde que es importante cerrar siempre las conexiones; de esta manera se evita que las conexiones que no se utilizan consuman recursos.

In [19]:
ibm_db.close(conn)

True

## Resumen 

En este tutorial, establecimos una conexión con una instancia de base de datos de Db2 Warehouse on Cloud desde una libreta de Python con la API de ibm_db. También creamos una tabla e insertamos algunas filas de datos en ella; luego, consultamos los datos. Por último, importamos estos datos a un DataFrame de Pandas. 

Derechos de autor &copy; 2018 [cognitiveclass.ai](cognitiveclass.ai?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). Esta cuaderno y su código fuente se publican bajo los términos de la [Licencia de MIT](https://bigdatauniversity.com/mit-license/).